In [ ]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from tqdm import tqdm

In [ ]:
base_url = "https://www.trustpilot.com"

In [ ]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [ ]:
#fetching sub-categories urls:

data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class': 'subCategory___BRUDy'}):
    name = category.find('h3', {'class': 'subCategoryHeader___36ykD'}).text
    name = name.strip()
    data[name] = {}
    
    sub_categories = category.find('div', {'class': 'subCategoryList___r67Qj'})
    for sub_category in sub_categories.findAll('div', {'class': 'subCategoryItem___3ksKz'}):
        sub_category_name = sub_category.find('a', {'class': 'navigation___2Efid'}).text 
        sub_category_uri = sub_category.find('a', {'class': 'navigation___2Efid'})['href'] 
        data[name][sub_category_name] = sub_category_uri

In [ ]:
len(data)

In [ ]:
data

In [ ]:
# data = dict((key,value) for key, value in data.items() if key == 'Events & Entertainment')

In [ ]:
data.pop("Events & Entertainment")
data.pop("Home Services")

In [ ]:
data

In [ ]:
# fetch company urls referenced in a given subcategory:

def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('(//div[@class="businessUnitCardsContainer___Qhix1"])/a')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    return dedup_urls

In [ ]:
# if a pagination exists on the page:

def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

In [ ]:
# initializing Selenium with a headless firefoxdriver:

options = Options()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

# profile = webdriver.FirefoxProfile()
# profile.set_preference("browser.cache.disk.enable", False)
# profile.set_preference("browser.cache.memory.enable", False)
# profile.set_preference("browser.cache.offline.enable", False)
# profile.set_preference("network.http.use-cache", False)

driver = webdriver.Firefox(executable_path=r'driver\geckodriver.exe', options=options)

timeout = 3

In [ ]:
# scraping

company_urls = {}
count = 1
for category in tqdm(data):
    for sub_category in tqdm(data[category], leave=False):
        company_urls[sub_category] = []
        print("Category is: ", category)
        print("Sub-Category is: ", sub_category)
        url = base_url + data[category][sub_category] + "?numberofreviews=0&status=all&timeperiod=0"
        driver.get(url)
        print("URL is: ", url)
        
#         current_url =  driver.current_url
#         print("Current URL is: ", current_url)
        
#         driver.implicitly_wait(10)
        
#         if data[category][sub_category] != ((current_url.split('.com'))[1].split('?')[0]):
#             print("Change in Url is detected")
#             data[category][sub_category] = ((current_url.split('.com'))[1].split('?')[0])
#             sub_category = ((current_url.split('/categories/'))[1].split('?')[0])
#             print("Now data[category][sub_category] is : ", data[category][sub_category])
            
        try:
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'wrapper___2rOTx'))
            
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
    
        next_page = True
        c = 1

        while next_page:
            #driver.implicitly_wait(10)
            extracted_company_urls = extract_company_urls_form_page()
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            count = count + 1
            
            if count > 200:
                driver.close()
                driver = webdriver.Firefox(executable_path=r'driver\geckodriver.exe', options=options)
                count = 1
                print("\nClosed and then Up Again\n")
            
            if next_page:
                c += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&status=all&timeperiod=0" + f'&page={c}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'wrapper___2rOTx'))
                    # wrapper___2rOTx
                    WebDriverWait(driver, timeout).until(element_present)
                    
                    
                except:
                    pass

In [ ]:
driver.quit()

In [ ]:
with open('./exports/company_urls_en.json', 'w') as f:
    json.dump(company_urls, f)

In [ ]:
consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('./exports/consolidate_company_urls.csv', index=False)

In [ ]:
df_consolidated_data.head()